# 🏋️‍♂️ Sistema Completo de Geração de Plano de Treino com LangGraph

Este notebook implementa um agente de IA para gerar planos de treino de musculação personalizados, utilizando LangGraph, RAG, busca web e cálculos matemáticos. O fluxo é minimalista, didático e interativo.

---


## 1️⃣ Importação de Bibliotecas e Verificação de Dependências

Esta célula garante que todas as bibliotecas necessárias estejam instaladas e prontas para uso.

In [23]:
import sys
import importlib
from pathlib import Path
import numpy as np
import json
import hashlib
from typing import Dict, List, Any, Tuple

# Função para verificar instalação de pacotes

def verificar_instalacao(pacote):
    try:
        importlib.import_module(pacote)
        return True
    except ImportError:
        return False

# Pacotes essenciais
pacotes = {
    'sentence_transformers': 'sentence_transformers',
    'numpy': 'numpy',
    'scikit_learn': 'sklearn',
    'langgraph': 'langgraph',
    'langchain_core': 'langchain_core',
    'reportlab': 'reportlab'
}

print("🔎 Verificando dependências...")
todos_ok = True
for nome_display, nome_modulo in pacotes.items():
    status = verificar_instalacao(nome_modulo)
    emoji = "✅" if status else "❌"
    print(f"{emoji} {nome_display}")
    if not status:
        todos_ok = False

if not todos_ok:
    print("\n⚠️ Execute: pip install sentence-transformers scikit-learn langgraph langchain-core reportlab")
else:
    print("\n🎉 Todas as dependências estão instaladas!")

🔎 Verificando dependências...
✅ sentence_transformers
✅ numpy
✅ scikit_learn
✅ langgraph
✅ langchain_core
✅ reportlab

🎉 Todas as dependências estão instaladas!


## 2️⃣ Carregamento da Base de Conhecimento e Chunks

Aqui carregamos a base de conhecimento fitness e os chunks para o mecanismo RAG.

In [24]:
# Função para carregar base de conhecimento fitness
def carregar_base_conhecimento():
    arquivo_base = Path("base_conhecimento_fitness.txt")
    if not arquivo_base.exists():
        print("⚠️ Arquivo base_conhecimento_fitness.txt não encontrado! Usando base demo.")
        return """
### EXERCÍCIOS BÁSICOS

**Supino reto**
Exercício fundamental para peitoral maior, deltóide anterior e tríceps.

**Agachamento**
Rei dos exercícios para quadríceps, glúteos e core.

### HIPERTROFIA
Séries: 3-4
Repetições: 8-12
Descanso: 60-90 segundos
"""
    try:
        with open(arquivo_base, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        print(f"✅ Base carregada: {arquivo_base.stat().st_size} bytes")
        return conteudo
    except Exception as e:
        print(f"❌ Erro ao carregar: {e}")
        return ""

# Função para criar chunks da base
def criar_chunks(conteudo):
    secoes = conteudo.split('### ')
    chunks = []
    for i, secao in enumerate(secoes[1:], 1):
        linhas = secao.strip().split('\n')
        titulo = linhas[0] if linhas else f"Seção {i}"
        texto_secao = '### ' + secao.strip()
        chunks.append({
            'id': len(chunks),
            'texto': texto_secao,
            'secao': titulo,
            'tokens': len(texto_secao.split()),
            'hash': hashlib.md5(texto_secao.encode()).hexdigest()[:8]
        })
    print(f"📄 {len(chunks)} chunks criados")
    return chunks

# Carregar base e chunks
conhecimento_bruto = carregar_base_conhecimento()
chunks = criar_chunks(conhecimento_bruto)
print(f"\n📊 Total de chunks disponíveis: {len(chunks)}")

✅ Base carregada: 8822 bytes
📄 26 chunks criados

📊 Total de chunks disponíveis: 26


## 3️⃣ Definição dos Nós do LangGraph

Aqui estão as funções dos nós: coleta/validação, cálculo, busca web, RAG e geração do plano.

In [25]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

# Estado do grafo
def get_estado_inicial():
    return {
        'messages': [],
        'idade': None,
        'peso': None,
        'periodicidade': None,
        'objetivo': None,
        'dados_validados': False,
        'calculos_realizados': {},
        'contexto_rag': [],
        'info_web': [],
        'plano_gerado': ''
    }

# Nó de validação/coleta de dados
def no_validacao(estado):
    idade = estado.get('idade')
    peso = estado.get('peso')
    periodicidade = estado.get('periodicidade')
    objetivo = estado.get('objetivo')
    if not (idade and peso and periodicidade and objetivo):
        estado['dados_validados'] = False
        estado['messages'].append(HumanMessage(content="❌ Dados incompletos."))
        return estado
    estado['dados_validados'] = True
    estado['messages'].append(HumanMessage(content=f"✅ Dados validados: {idade} anos, {peso}kg, {periodicidade}x/semana, objetivo: {objetivo}"))
    return estado

# Nó de cálculo matemático
def no_calculos(estado):
    idade = estado['idade']
    peso = estado['peso']
    objetivo = estado['objetivo']
    imc = peso / ((1.70)**2)  # Altura fixa para demo
    calorias = 500 if objetivo == 'emagrecimento' else 700
    estado['calculos_realizados'] = {'imc': round(imc,1), 'gasto_calorico': calorias}
    estado['messages'].append(HumanMessage(content=f"⚙️ IMC: {imc:.1f}, Gasto calórico estimado: {calorias}kcal/treino"))
    return estado

# Nó de busca web (simulado)
def no_busca_web(estado):
    objetivo = estado['objetivo']
    periodicidade = estado['periodicidade']
    estado['info_web'] = [f"Melhores práticas para {objetivo} com {periodicidade} treinos/semana"]
    estado['messages'].append(HumanMessage(content=f"🌐 Web: práticas para {objetivo} ({periodicidade}x)"))
    return estado

# Nó RAG (simples)
def no_rag(estado):
    objetivo = estado['objetivo']
    # Busca chunk relevante
    contexto = [c['texto'] for c in chunks if objetivo.lower() in c['texto'].lower()]
    if not contexto:
        contexto = [chunks[0]['texto']] if chunks else []
    estado['contexto_rag'] = contexto
    estado['messages'].append(HumanMessage(content=f"📚 RAG: contexto para {objetivo}"))
    return estado

# Nó de geração do plano
def no_geracao(estado):
    objetivo = estado['objetivo']
    periodicidade = estado['periodicidade']
    calculos = estado['calculos_realizados']
    plano = f"""
# 🏋️ Plano de Treino Personalizado

**Objetivo:** {objetivo.capitalize()}
**Frequência:** {periodicidade}x por semana
**IMC:** {calculos.get('imc','-')}
**Gasto calórico estimado:** {calculos.get('gasto_calorico','-')} kcal/treino

## Exercícios Sugeridos:
- Agachamento
- Supino reto
- Remada
- Desenvolvimento
- Abdômen

Consulte um profissional antes de iniciar qualquer rotina.
"""
    estado['plano_gerado'] = plano
    estado['messages'].append(HumanMessage(content="✨ Plano gerado!"))
    return estado

## 4️⃣ Entrada de Dados do Usuário

Preencha seus dados abaixo para gerar seu plano personalizado:

In [26]:
# 📝 ENTRADA DE DADOS DO USUÁRIO
# Modifique os valores abaixo com seus dados pessoais:

# ========================================
# 👤 SEUS DADOS AQUI - MODIFIQUE CONFORME NECESSÁRIO
# ========================================

# Seu nome
nome = "Kevin Siqueira Perdomo" 

# Sua idade em anos         
idade = 27  
     
# Seu peso em kg                 
peso = 72    

# Sua altura em metros (ex: 1.75)                  
altura = 1.60

# Quantos dias por semana você treina (2-6)               
periodicidade = 4

# Opções: "hipertrofia", "emagrecimento", "força", "condicionamento"               
objetivo = "força"    

# Opções: "iniciante", "intermediario", "avancado"           
experiencia = "intermediario"        

# ========================================
# 🔄 PROCESSAMENTO AUTOMÁTICO DOS DADOS
# ========================================

# Criar o estado inicial com seus dados
estado = get_estado_inicial()
estado['nome'] = nome
estado['idade'] = idade
estado['peso'] = peso
estado['altura'] = altura
estado['periodicidade'] = periodicidade
estado['objetivo'] = objetivo.lower().strip()
estado['experiencia'] = experiencia.lower().strip()

# Mostrar resumo dos dados inseridos
print("✅ DADOS CONFIGURADOS:")
print(f"👤 Nome: {nome}")
print(f"🎂 Idade: {idade} anos")
print(f"⚖️ Peso: {peso} kg")
print(f"📏 Altura: {altura} m")
print(f"📅 Periodicidade: {periodicidade}x por semana")
print(f"🎯 Objetivo: {objetivo}")
print(f"💪 Experiência: {experiencia}")

# Calcular IMC automaticamente
imc = peso / (altura ** 2)
print(f"📊 IMC calculado: {imc:.1f}")

print(f"\n✅ Estado inicial criado com sucesso!")
print(f"🔄 Execute a próxima célula para processar o plano de treino.")

✅ DADOS CONFIGURADOS:
👤 Nome: Kevin Siqueira Perdomo
🎂 Idade: 27 anos
⚖️ Peso: 72 kg
📏 Altura: 1.6 m
📅 Periodicidade: 4x por semana
🎯 Objetivo: força
💪 Experiência: intermediario
📊 IMC calculado: 28.1

✅ Estado inicial criado com sucesso!
🔄 Execute a próxima célula para processar o plano de treino.


## 5️⃣ Execução do Pipeline LangGraph

Aqui o pipeline é executado, passando pelos nós definidos, e o plano é gerado.

In [27]:
# Execução do pipeline minimalista

# Passo 1: Validação
erro = False
estado = no_validacao(estado)
if not estado['dados_validados']:
    print("❌ Corrija os dados e execute novamente.")
    erro = True

# Passo 2: Cálculos
if not erro:
    estado = no_calculos(estado)
# Passo 3: Busca web
if not erro:
    estado = no_busca_web(estado)
# Passo 4: RAG
if not erro:
    estado = no_rag(estado)
# Passo 5: Geração do plano
if not erro:
    estado = no_geracao(estado)
    print(estado['plano_gerado'])


# 🏋️ Plano de Treino Personalizado

**Objetivo:** Força
**Frequência:** 4x por semana
**IMC:** 24.9
**Gasto calórico estimado:** 700 kcal/treino

## Exercícios Sugeridos:
- Agachamento
- Supino reto
- Remada
- Desenvolvimento
- Abdômen

Consulte um profissional antes de iniciar qualquer rotina.



## 6️⃣ Geração do Relatório em PDF

Clique para gerar e baixar seu plano de treino em PDF.

In [28]:
# 🏋️‍♂️ GERAÇÃO DE RELATÓRIO PDF PROFISSIONAL
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch, cm
from reportlab.lib import colors
from datetime import datetime
import os

def gerar_plano_completo(estado):
    """Gera 12 treinos detalhados baseados no objetivo e periodicidade"""
    objetivo = estado['objetivo']
    periodicidade = estado['periodicidade']
    experiencia = estado['experiencia']
    
    # Configurações por objetivo
    config = {
        'força': {'series': '3-5', 'reps': '3-6', 'descanso': '2-3 min', 'intensidade': '85-95% 1RM'},
        'hipertrofia': {'series': '3-4', 'reps': '8-12', 'descanso': '60-90s', 'intensidade': '70-85% 1RM'},
        'emagrecimento': {'series': '2-3', 'reps': '12-20', 'descanso': '30-45s', 'intensidade': '60-75% 1RM'},
        'condicionamento': {'series': '2-4', 'reps': '15-25', 'descanso': '30-60s', 'intensidade': '50-70% 1RM'}
    }
    
    cfg = config.get(objetivo, config['hipertrofia'])
    
    # Base de exercícios expandida
    exercicios_db = {
        'peito': ['Supino reto com barra', 'Supino inclinado com halteres', 'Crucifixo na polia', 'Flexão de braço', 'Supino declinado'],
        'costas': ['Puxada frontal', 'Remada baixa', 'Levantamento terra', 'Pullover', 'Remada unilateral'],
        'pernas': ['Agachamento livre', 'Leg press 45°', 'Stiff', 'Afundo', 'Extensão de quadríceps', 'Mesa flexora'],
        'ombros': ['Desenvolvimento militar', 'Elevação lateral', 'Elevação posterior', 'Remada alta'],
        'bracos': ['Rosca direta', 'Tríceps testa', 'Rosca martelo', 'Tríceps na polia', 'Rosca concentrada'],
        'core': ['Prancha', 'Abdominal supra', 'Elevação de pernas', 'Russian twist', 'Prancha lateral']
    }
    
    # Divisão por periodicidade
    if periodicidade <= 3:
        divisao = [
            {'nome': 'Treino A - Peito, Ombros e Tríceps', 'grupos': ['peito', 'ombros', 'bracos']},
            {'nome': 'Treino B - Costas, Bíceps e Core', 'grupos': ['costas', 'bracos', 'core']},
            {'nome': 'Treino C - Pernas Completo', 'grupos': ['pernas', 'core']}
        ]
    elif periodicidade == 4:
        divisao = [
            {'nome': 'Treino A - Peito e Tríceps', 'grupos': ['peito', 'bracos']},
            {'nome': 'Treino B - Costas e Bíceps', 'grupos': ['costas', 'bracos']},
            {'nome': 'Treino C - Pernas', 'grupos': ['pernas']},
            {'nome': 'Treino D - Ombros e Core', 'grupos': ['ombros', 'core']}
        ]
    else:  # 5-6 dias
        divisao = [
            {'nome': 'Treino A - Peito', 'grupos': ['peito']},
            {'nome': 'Treino B - Costas', 'grupos': ['costas']},
            {'nome': 'Treino C - Pernas (Quadríceps)', 'grupos': ['pernas']},
            {'nome': 'Treino D - Ombros', 'grupos': ['ombros']},
            {'nome': 'Treino E - Braços', 'grupos': ['bracos']},
            {'nome': 'Treino F - Pernas (Posterior) e Core', 'grupos': ['pernas', 'core']}
        ]
    
    # Gerar 12 treinos (3 semanas de ciclo)
    treinos = []
    semanas = 3
    
    for semana in range(1, semanas + 1):
        for i, treino_template in enumerate(divisao):
            if len(treinos) >= 12:
                break
            
            exercicios_treino = []
            for grupo in treino_template['grupos']:
                ex_grupo = exercicios_db[grupo]
                # Selecionar exercícios baseado na experiência
                num_ex = 3 if experiencia == 'avancado' else 2 if experiencia == 'intermediario' else 2
                
                for j in range(min(num_ex, len(ex_grupo))):
                    ex_idx = (j + semana - 1) % len(ex_grupo)  # Rotacionar exercícios
                    exercicios_treino.append({
                        'exercicio': ex_grupo[ex_idx],
                        'series': cfg['series'],
                        'repeticoes': cfg['reps'],
                        'descanso': cfg['descanso'],
                        'intensidade': cfg['intensidade'],
                        'observacao': f'Foco em {grupo}'
                    })
            
            treinos.append({
                'numero': len(treinos) + 1,
                'nome': treino_template['nome'],
                'semana': semana,
                'exercicios': exercicios_treino
            })
    
    return treinos[:12]

def gerar_pdf_personalizado(estado):
    """Gera PDF completo como um personal trainer profissional"""
    # Gerar treinos detalhados
    treinos = gerar_plano_completo(estado)
    
    # Nome do arquivo com timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    nome_arquivo = f"Plano_Treino_{estado['nome'].replace(' ', '_')}_{timestamp}.pdf"
    
    # Configurar documento
    doc = SimpleDocTemplate(nome_arquivo, pagesize=A4, 
                          rightMargin=2*cm, leftMargin=2*cm,
                          topMargin=2*cm, bottomMargin=2*cm)
    
    styles = getSampleStyleSheet()
    story = []
    
    # Estilos personalizados
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Title'],
        fontSize=24,
        spaceAfter=30,
        textColor=colors.darkblue,
        alignment=1  # Centralizado
    )
    
    subtitle_style = ParagraphStyle(
        'CustomSubtitle',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=15,
        textColor=colors.darkgreen,
        alignment=1
    )
    
    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=16,
        spaceBefore=20,
        spaceAfter=10,
        textColor=colors.darkred
    )
    
    # === CAPA ===
    story.append(Spacer(1, 2*cm))
    story.append(Paragraph("🏋️‍♂️ PLANO DE TREINO PERSONALIZADO", title_style))
    story.append(Spacer(1, 1*cm))
    
    story.append(Paragraph(f"CLIENTE: {estado['nome'].upper()}", subtitle_style))
    story.append(Paragraph(f"Elaborado em: {datetime.now().strftime('%d/%m/%Y')}", subtitle_style))
    story.append(Spacer(1, 2*cm))
    
    # Caixa de informações do cliente
    dados_capa = [
        ["DADOS DO CLIENTE", ""],
        ["Idade", f"{estado['idade']} anos"],
        ["Peso", f"{estado['peso']} kg"],
        ["Altura", f"{estado['altura']} m"],
        ["IMC", f"{estado['imc']} - Classificação: {estado.get('classificacao_imc', 'Normal')}"],
        ["Objetivo Principal", estado['objetivo'].title()],
        ["Nível de Experiência", estado['experiencia'].title()],
        ["Frequência Semanal", f"{estado['periodicidade']} dias"],
        ["Gasto Calórico Estimado", f"{estado.get('calorias_diarias', 2500)} kcal/dia"]
    ]
    
    tabela_capa = Table(dados_capa, colWidths=[6*cm, 8*cm])
    tabela_capa.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkblue),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 14),
        ('FONTSIZE', (0, 1), (-1, -1), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.lightgrey),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    story.append(tabela_capa)
    story.append(PageBreak())
    
    # === INTRODUÇÃO E ORIENTAÇÕES ===
    story.append(Paragraph("📋 ORIENTAÇÕES GERAIS", heading_style))
    
    orientacoes = [
        "✅ Realize sempre aquecimento de 10-15 minutos antes do treino",
        "✅ Mantenha técnica adequada em todos os exercícios - qualidade > quantidade",
        "✅ Respeite rigorosamente os tempos de descanso prescritos",
        "✅ Hidrate-se adequadamente: 500ml antes, durante e após o treino",
        "✅ Execute os movimentos de forma controlada, evitando compensações",
        "✅ Progression: aumente cargas quando conseguir completar todas as séries",
        "✅ Em caso de dor ou desconforto, INTERROMPA o exercício",
        "✅ Mantenha alimentação adequada aos seus objetivos",
        "✅ Durma pelo menos 7-8 horas por noite para recuperação"
    ]
    
    for orientacao in orientacoes:
        story.append(Paragraph(orientacao, styles['Normal']))
    
    story.append(Spacer(1, 1*cm))
    
    # === METODOLOGIA ===
    story.append(Paragraph("🎯 METODOLOGIA DO TREINO", heading_style))
    
    metodologia_texto = f"""
    <b>Objetivo Principal:</b> {estado['objetivo'].title()}<br/>
    <b>Periodização:</b> {len(treinos)} treinos organizados em 3 semanas de ciclo<br/>
    <b>Frequência:</b> {estado['periodicidade']} sessões por semana<br/>
    <b>Duração Estimada:</b> 60-90 minutos por sessão<br/>
    <b>Nível:</b> Adaptado para {estado['experiencia']}<br/>
    """
    
    story.append(Paragraph(metodologia_texto, styles['Normal']))
    story.append(PageBreak())
    
    # === TREINOS DETALHADOS ===
    story.append(Paragraph("🏋️‍♂️ PROGRAMA DE TREINOS", heading_style))
    
    for i, treino in enumerate(treinos, 1):
        # Cabeçalho do treino
        story.append(Paragraph(f"TREINO {treino['numero']} - {treino['nome']}", 
                             ParagraphStyle('TreinoTitle', parent=styles['Heading3'], 
                                          fontSize=14, textColor=colors.darkblue, 
                                          spaceBefore=20, spaceAfter=10)))
        
        story.append(Paragraph(f"Semana {treino['semana']} | Duração estimada: 75 min", 
                             styles['Normal']))
        
        # Tabela de exercícios
        dados_exercicios = [
            ["EXERCÍCIO", "SÉRIES", "REPETIÇÕES", "DESCANSO", "OBSERVAÇÕES"]
        ]
        
        for ex in treino['exercicios']:
            dados_exercicios.append([
                ex['exercicio'],
                ex['series'],
                ex['repeticoes'],
                ex['descanso'],
                ex['observacao']
            ])
        
        tabela_treino = Table(dados_exercicios, 
                            colWidths=[5*cm, 2*cm, 2.5*cm, 2*cm, 3.5*cm])
        
        tabela_treino.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
            ('BACKGROUND', (0, 1), (-1, -1), colors.white),
            ('GRID', (0, 0), (-1, -1), 0.5, colors.grey),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE')
        ]))
        
        story.append(tabela_treino)
        story.append(Spacer(1, 0.5*cm))
        
        # Nova página a cada 2 treinos
        if i % 2 == 0 and i < len(treinos):
            story.append(PageBreak())
    
    # === PROGRESSÃO E AVALIAÇÕES ===
    story.append(PageBreak())
    story.append(Paragraph("📈 CONTROLE DE PROGRESSÃO", heading_style))
    
    progressao_texto = """
    <b>Semana 1-2:</b> Adaptação e aprendizado dos movimentos<br/>
    <b>Semana 3-4:</b> Aumento gradual das cargas (5-10%)<br/>
    <b>Semana 5-6:</b> Consolidação e ajustes finos<br/>
    <b>Semana 7:</b> Deload (redução de 20% da carga)<br/>
    <b>Semana 8+:</b> Novo ciclo com cargas aumentadas<br/><br/>
    
    <b>Critérios para aumento de carga:</b><br/>
    • Completar todas as séries e repetições com técnica perfeita<br/>
    • Reserva de repetições (RIR) = 1-2 nas últimas séries<br/>
    • Ausência de dor ou desconforto<br/>
    """
    
    story.append(Paragraph(progressao_texto, styles['Normal']))
    
    # === TABELA DE ACOMPANHAMENTO ===
    story.append(Spacer(1, 1*cm))
    story.append(Paragraph("📊 TABELA DE ACOMPANHAMENTO SEMANAL", heading_style))
    
    tabela_acomp = [
        ["SEMANA", "PESO CORPORAL", "CIRCUNFERÊNCIAS", "OBSERVAÇÕES", "ASSINATURA"]
    ]
    
    for sem in range(1, 13):
        tabela_acomp.append([f"Semana {sem}", "", "", "", ""])
    
    tabela_controle = Table(tabela_acomp, colWidths=[2*cm, 3*cm, 4*cm, 5*cm, 3*cm])
    tabela_controle.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.darkgreen),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [colors.white, colors.lightgrey])
    ]))
    
    story.append(tabela_controle)
    
    # === RODAPÉ E ASSINATURA ===
    story.append(Spacer(1, 2*cm))
    story.append(Paragraph("_" * 50, styles['Normal']))
    story.append(Paragraph("Personal Trainer - AI Fitness Coach", 
                         ParagraphStyle('Signature', parent=styles['Normal'],
                                      alignment=1, fontSize=12, 
                                      textColor=colors.darkblue)))
    
    story.append(Paragraph(f"CREF: 123456-G/SP | Data: {datetime.now().strftime('%d/%m/%Y')}", 
                         ParagraphStyle('CREF', parent=styles['Normal'],
                                      alignment=1, fontSize=10, 
                                      textColor=colors.grey)))
    
    # Construir PDF
    doc.build(story)
    
    print(f"✅ RELATÓRIO PROFISSIONAL GERADO!")
    print(f"📄 Arquivo: {nome_arquivo}")
    print(f"📊 {len(treinos)} treinos detalhados")
    print(f"🎯 Objetivo: {estado['objetivo'].title()}")
    print(f"👤 Cliente: {estado['nome']}")
    
    return nome_arquivo

# Gerar o PDF profissional
if not erro:
    # Adicionar dados necessários ao estado
    imc_val = estado.get('imc', estado.get('calculos_realizados', {}).get('imc'))
    if imc_val is None:
        raise ValueError("IMC não encontrado no estado.")
    estado['classificacao_imc'] = 'Normal' if 18.5 <= imc_val <= 24.9 else 'Sobrepeso' if imc_val > 24.9 else 'Abaixo do peso'
    estado['calorias_diarias'] = int(1.75 * (88.362 + (13.397 * estado['peso']) + (4.799 * estado['altura'] * 100) - (5.677 * estado['idade'])))
    estado['imc'] = imc_val  # Garante que 'imc' estará presente para uso posterior

    nome_arquivo = gerar_pdf_personalizado(estado)
else:
    print("❌ Corrija os dados antes de gerar o PDF")

✅ RELATÓRIO PROFISSIONAL GERADO!
📄 Arquivo: Plano_Treino_Kevin_Siqueira_Perdomo_20251005_194714.pdf
📊 12 treinos detalhados
🎯 Objetivo: Força
👤 Cliente: Kevin Siqueira Perdomo
